In [ ]:
#analyse data
#check for missing value
#check for duplicte value nd decide if to include in your single data souce of truth
#check other data sources that needs to join the sas data
#create single data source of truth
#create fact and dimension table for anlytics purpose (add check for duplicate value)

In [ ]:
###tables to get.
#immigration table(just like songsplay table)
#loction (us destination cities)
#airports
#departure country
#date - time table
#temperature table


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

from datetime import datetime, timedelta

from pyspark.sql import Window

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

import re

In [2]:
spark = SparkSession \
        .builder \
        .appName("capstone project") \
        .getOrCreate()

In [19]:
fname = 'source_data/sas_data'

In [20]:
global_temp_path = '../../data2/GlobalLandTemperaturesByCity.csv'

In [21]:
pd.set_option('display.max_columns', None)

In [22]:
df = spark.read.parquet(fname)

In [23]:
df_country_code = spark.read.csv("source_data/country_code_processed.csv", header = True)

In [24]:
df_country_code.limit(10).toPandas().head()

,country_code,country_name
0,582,"MEXICO Air Sea, and Not Reported (I-94, no l..."
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA


In [9]:
df.limit(10).toPandas().head(20)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,5748517.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,CA,20582.0,40.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1976.0,10292016,F,None,QF,9.495387e+10,00011,B1
1,5748518.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,NV,20591.0,32.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1984.0,10292016,F,None,VA,9.495562e+10,00007,B1
2,5748519.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20582.0,29.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1987.0,10292016,M,None,DL,9.495641e+10,00040,B1
3,5748520.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,29.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1987.0,10292016,F,None,DL,9.495645e+10,00040,B1
4,5748521.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,28.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1988.0,10292016,M,None,DL,9.495639e+10,00040,B1
5,5748522.0,2016.0,4.0,245.0,464.0,HHW,20574.0,1.0,HI,20579.0,57.0,2.0,1.0,20160430,ACK,None,G,O,None,M,1959.0,10292016,M,None,NZ,9.498180e+10,00010,B2
6,5748523.0,2016.0,4.0,245.0,464.0,HHW,20574.0,1.0,HI,20586.0,66.0,2.0,1.0,20160430,ACK,None,G,O,None,M,1950.0,10292016,F,None,NZ,9.497969e+10,00010,B2
7,5748524.0,2016.0,4.0,245.0,464.0,HHW,20574.0,1.0,HI,20586.0,41.0,2.0,1.0,20160430,ACK,None,G,O,None,M,1975.0,10292016,F,None,NZ,9.497975e+10,00010,B2
8,5748525.0,2016.0,4.0,245.0,464.0,HOU,20574.0,1.0,FL,20581.0,27.0,2.0,1.0,20160430,ACK,None,G,O,None,M,1989.0,10292016,M,None,NZ,9.497325e+10,00028,B2
9,5748526.0,2016.0,4.0,245.0,464.0,LOS,20574.0,1.0,CA,20581.0,26.0,2.0,1.0,20160430,ACK,None,G,O,None,M,1990.0,10292016,F,None,NZ,9.501355e+10,00002,B2


In [ ]:
#create a normalized star schema and a completely de-normalised master transaction table
#import country file generated from label description file - done
#write date function for arrical and departure date - done
#convert i94yr and i94month as well as i94res(country where the immigrant traveled from) and i94cit (country of origin)to int - done
#format date add to (dtaddto) and date ad file(dtadfile) to yyyy-mm-dd format for uniforminty - done
#consider creating a date table
#


In [10]:
#df_immigration.arrdate conversion to datetime

def fn_format_datetime(x):
    try:
        start = datetime(1960, 1, 1)
        return start + timedelta(days=int(x))
    except:
        return None
    
#udf_datetime_conversion = udf(lambda x: convert_datetime(x), T.DateType())
#...In your Spark code
#spark...load the data etc.... \
 #   .withColumn("arrival_date", udf_datetime_from_sas("arrdate")) \
  #  .withColumn("departure_date", udf_datetime_from_sas("depdate"))

In [11]:
spark.udf.register("format_datetime", fn_format_datetime, T.DateType())

<function __main__.fn_format_datetime(x)>

In [12]:
df.createOrReplaceTempView("immigration_data")

In [13]:
immigration_table = spark.sql(
    """
    select  cast(cicid as int) as cicid
            ,cast(i94yr as int) as i94yr
            ,cast(i94mon as int) as i94mon
            ,cast(i94cit as int) as i94cit
            ,cast(i94res as int) as i94res
            ,i94port
            ,format_datetime(cast(arrdate as int)) as arrival_date
            ,cast(i94mode as int) as i94mode
            ,i94addr
            ,format_datetime(cast(depdate as int)) as depature_date
            ,cast(i94bir as int) as i94bir
            ,cast(i94visa as int) as i94visa
            ,cast(count as int) as count
            ,to_date(cast(UNIX_TIMESTAMP(dtadfile, 'MMddyyyy') AS TIMESTAMP)) AS dtadfile
            ,visapost
            ,occup
            ,entdepa
            ,entdepd
            ,entdepu
            ,matflag
            ,cast(biryear as int) as biryear
            ,to_date(cast(UNIX_TIMESTAMP(dtaddto, 'MMddyyyy') AS TIMESTAMP)) AS dtaddto
            ,gender
            ,insnum
            ,airline
            ,cast(admnum as decimal(16,8)) as admnum
            ,fltno
            ,visatype
            
    From immigration_data
    
    """

) 

In [14]:
immigration_table.limit(10).toPandas().head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrival_date,i94mode,i94addr,depature_date,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,5748517,2016,4,245,438,LOS,2016-04-30,1,CA,2016-05-08,40,1,1,None,SYD,None,G,O,None,M,1976,2016-10-29,F,None,QF,None,00011,B1
1,5748518,2016,4,245,438,LOS,2016-04-30,1,NV,2016-05-17,32,1,1,None,SYD,None,G,O,None,M,1984,2016-10-29,F,None,VA,None,00007,B1
2,5748519,2016,4,245,438,LOS,2016-04-30,1,WA,2016-05-08,29,1,1,None,SYD,None,G,O,None,M,1987,2016-10-29,M,None,DL,None,00040,B1
3,5748520,2016,4,245,438,LOS,2016-04-30,1,WA,2016-05-14,29,1,1,None,SYD,None,G,O,None,M,1987,2016-10-29,F,None,DL,None,00040,B1
4,5748521,2016,4,245,438,LOS,2016-04-30,1,WA,2016-05-14,28,1,1,None,SYD,None,G,O,None,M,1988,2016-10-29,M,None,DL,None,00040,B1


In [16]:
immigration_table.where(immigration_table.arrival_date.isNull()).limit(10).toPandas().head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrival_date,i94mode,i94addr,depature_date,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype


In [33]:
df_immigration = spark.read.csv("immigration_data_sample.csv", header = True)

In [26]:
df_cities = spark.read.option("delimiter", ";").csv("source_data/us-cities-demographics.csv", header = True)

In [16]:
df_airport = spark.read.csv("source_data/airport-codes_csv.csv", header = True)

In [17]:
df_globe_temp = spark.read.csv(global_temp_path, header = True)

In [27]:
#additional data (city and state code table)
df_city_state_code = spark.read.csv("source_data/us_cities_processed.csv", header = True)

In [60]:
df_immigration.limit(10).toPandas().head() #.where(df_immigration.i94addr == 'AK')

,_c0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,None,None,G,O,None,M,1955.0,07202016,F,None,JL,56582674633.0,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,None,G,R,None,M,1990.0,10222016,M,None,*GA,94361995930.0,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,None,None,G,O,None,M,1940.0,07052016,M,None,LH,55780468433.0,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,None,G,O,None,M,1991.0,10272016,M,None,QR,94789696030.0,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,None,None,Z,K,None,M,1997.0,07042016,F,None,None,42322572633.0,LAND,WT


In [40]:
df_cities.where(df_cities.City == 'Bk').limit(10).toPandas().head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,White,79582
1,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,Asian,14636
2,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,American Indian and Alaska Native,1639
3,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,Hispanic or Latino,22089
4,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,Black or African-American,4729


In [41]:
df_cities2 = df_cities.select([F.col(col).alias(col.replace(' ', '_')) for col in df_cities.columns])

In [42]:
df_cities2 = df_cities2.select([F.col(col).alias(col.replace('-', '_')) for col in df_cities2.columns])

In [43]:
df_cities2.where(df_cities2.City == 'Bk').limit(10).toPandas().head(10)

,City,State,Median_Age,Male_Population,Female_Population,Total_Population,Number_of_Veterans,Foreign_born,Average_Household_Size,State_Code,Race,Count
0,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,White,79582
1,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,Asian,14636
2,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,American Indian and Alaska Native,1639
3,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,Hispanic or Latino,22089
4,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,Black or African-American,4729


In [44]:
df_cities2.where((df_cities2.State_Code == 'CA') & (df_cities2.City == 'Bk')).limit(10).toPandas().head(5) 

,City,State,Median_Age,Male_Population,Female_Population,Total_Population,Number_of_Veterans,Foreign_born,Average_Household_Size,State_Code,Race,Count
0,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,White,79582
1,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,Asian,14636
2,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,American Indian and Alaska Native,1639
3,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,Hispanic or Latino,22089
4,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,Black or African-American,4729


In [46]:
df_cities2 = df_cities2.withColumn("city_state_code", F.concat_ws(('_'), F.col(("city")), F.col(("State_Code"))))

In [47]:
df_cities2.where(df_cities2.City == 'Bk').limit(10).toPandas().head()

,City,State,Median_Age,Male_Population,Female_Population,Total_Population,Number_of_Veterans,Foreign_born,Average_Household_Size,State_Code,Race,Count,city_state_code
0,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,White,79582,Bk_CA
1,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,Asian,14636,Bk_CA
2,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,American Indian and Alaska Native,1639,Bk_CA
3,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,Hispanic or Latino,22089,Bk_CA
4,Bk,California,37.0,50405,54909,105314,4048,32435,2.52,CA,Black or African-American,4729,Bk_CA


In [48]:
df_race = df_cities2.select(df_cities2.city_state_code, df_cities2.Race, df_cities2.Count.cast('int'))

In [49]:
df_race.show(5, truncate=False)

+-------------------+-------------------------+-----+
|city_state_code    |Race                     |Count|
+-------------------+-------------------------+-----+
|Silver Spring_MD   |Hispanic or Latino       |25924|
|Quincy_MA          |White                    |58723|
|Hoover_AL          |Asian                    |4759 |
|Rancho Cucamonga_CA|Black or African-American|24437|
|Newark_NJ          |White                    |76402|
+-------------------+-------------------------+-----+
only showing top 5 rows



In [50]:
df_race = df_race.groupBy("city_state_code").pivot("Race").sum("Count")

In [51]:
df_race.show(5)

+-------------------+---------------------------------+-----+-------------------------+------------------+------+
|    city_state_code|American Indian and Alaska Native|Asian|Black or African-American|Hispanic or Latino| White|
+-------------------+---------------------------------+-----+-------------------------+------------------+------+
|Huntington Beach_CA|                             2843|29095|                     4027|             42303|156071|
|          Clovis_CA|                             1876|14249|                     3434|             23744| 78029|
|     West Covina_CA|                              518|32716|                     3693|             58907| 48046|
|        Edinburg_TX|                             null|  812|                     null|             74698| 77560|
|   Santa Barbara_CA|                             1560| 4964|                     2013|             31102| 73659|
+-------------------+---------------------------------+-----+-------------------------+-

In [52]:
df_race = df_race.select([F.col(col).alias(col.replace(' ', '_')) for col in df_race.columns])

In [53]:
df_race = df_race.select([F.col(col).alias(col.replace('-', '_')) for col in df_race.columns])

In [54]:
df_race.show(5)

+-------------------+---------------------------------+-----+-------------------------+------------------+------+
|    city_state_code|American_Indian_and_Alaska_Native|Asian|Black_or_African_American|Hispanic_or_Latino| White|
+-------------------+---------------------------------+-----+-------------------------+------------------+------+
|Huntington Beach_CA|                             2843|29095|                     4027|             42303|156071|
|          Clovis_CA|                             1876|14249|                     3434|             23744| 78029|
|     West Covina_CA|                              518|32716|                     3693|             58907| 48046|
|        Edinburg_TX|                             null|  812|                     null|             74698| 77560|
|   Santa Barbara_CA|                             1560| 4964|                     2013|             31102| 73659|
+-------------------+---------------------------------+-----+-------------------------+-

In [33]:
df_city_state_code.limit(10).toPandas().head()

,city_code,city_name,state_code
0,ALC,ALCAN,AK
1,ANC,ANCHORAGE,AK
2,BAR,BAKER AAF - BAKER ISLAND,AK
3,DAC,DALTONS CACHE,AK
4,PIZ,DEW STATION PT LAY DEW,AK


In [34]:
df_city_state_code = df_city_state_code.withColumn("city_state_code", F.concat_ws(('_'), F.col(("city_name")), F.col(("state_code"))))

In [35]:
df_city_state_code.limit(10).toPandas().head()

,city_code,city_name,state_code,city_state_code
0,ALC,ALCAN,AK,ALCAN_AK
1,ANC,ANCHORAGE,AK,ANCHORAGE_AK
2,BAR,BAKER AAF - BAKER ISLAND,AK,BAKER AAF - BAKER ISLAND_AK
3,DAC,DALTONS CACHE,AK,DALTONS CACHE_AK
4,PIZ,DEW STATION PT LAY DEW,AK,DEW STATION PT LAY DEW_AK


In [36]:
df_cities2.createOrReplaceTempView("cities")

In [37]:
df_race.createOrReplaceTempView("race_table")

In [38]:
df_city_state_code.createOrReplaceTempView("city_code")

In [39]:
city_state_code = spark.sql(
    """
    select e.City
            ,e.State
            ,cc.city_code
            ,e.Median_Age
            ,cast(e.Male_Population as int) as Male_Population
            ,cast(e.Female_Population as int) as Female_Population
            ,cast(e.Total_Population as int) as Total_Population
            ,cast(e.Number_of_Veterans as int) as Number_of_Veterans
            ,cast(e.Foreign_born as int) as Foreign_born
            ,e.Average_Household_Size
            ,e.State_Code
            ,r.*
            
    from (select 
            City
            ,State
            ,city_state_code
            ,avg(Median_Age) as Median_Age
            ,sum(Male_Population) as Male_Population
            ,sum(Female_Population) as Female_Population
            ,sum(Total_Population) as Total_Population
            ,sum(Number_of_Veterans) as Number_of_Veterans
            ,sum(Foreign_born) as Foreign_born
            ,avg(Average_Household_Size) as Average_Household_Size
            ,State_Code
            --,r.*
            --,Race
            --,Count
            from cities
    group by city_state_code, City, State, State_Code ) as e
    left join race_table r on e.city_state_code = r.city_state_code
    left join city_code cc on lower(e.city_state_code) = lower(cc.city_state_code)
    
    """



)

In [44]:
city_state_code.where(city_state_code.city_code == 'PHI').toPandas().head()

,City,State,city_code,Median_Age,Male_Population,Female_Population,Total_Population,Number_of_Veterans,Foreign_born,Average_Household_Size,State_Code,city_state_code,American_Indian_and_Alaska_Native,Asian,Black_or_African_American,Hispanic_or_Latino,White
0,Philadelphia,Pennsylvania,PHI,34.1,3706350,4130860,7837210,309975,1026695,2.61,PA,Philadelphia_PA,17500,122721,691186,219038,688130


In [41]:
df_race = city_state_code.select()

In [42]:
city_code.show(5)

NameError: name 'city_code' is not defined

In [ ]:
cities.show(5)

In [45]:
df_airport.where((df_airport.iso_region == 'US-CA') & (df_airport.iata_code == 'SFR')).limit(20).toPandas().head(20)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,SFR,closed,San Fernando Airport,1168,NA,US,US-CA,Los Angeles,None,SFR,None,"-118.422, 34.289"


In [46]:
df_airport.where(df_airport.iata_code.isNull()).limit(20).toPandas().head(10)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,"-91.254898, 35.6087"
5,00AS,small_airport,Fulton Airport,1100,NA,US,US-OK,Alex,00AS,None,00AS,"-97.8180194, 34.9428028"
6,00AZ,small_airport,Cordes Airport,3810,NA,US,US-AZ,Cordes,00AZ,None,00AZ,"-112.16500091552734, 34.305599212646484"
7,00CA,small_airport,Goldstone /Gts/ Airport,3038,NA,US,US-CA,Barstow,00CA,None,00CA,"-116.888000488, 35.350498199499995"
8,00CL,small_airport,Williams Ag Airport,87,NA,US,US-CA,Biggs,00CL,None,00CL,"-121.763427, 39.427188"
9,00CN,heliport,Kitchen Creek Helibase Heliport,3350,NA,US,US-CA,Pine Valley,00CN,None,00CN,"-116.4597417, 32.7273736"


In [47]:
df3 = df_airport.withColumn("longitude", F.split(df_airport.coordinates, ",").getItem(0)).withColumn("latitude", F.split(df_airport.coordinates, ",").getItem(1))

In [48]:
df3.limit(20).toPandas().head(10)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,longitude,latitude
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125",-74.93360137939453,40.07080078125
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022",-101.473911,38.704022
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968",-151.695999146,59.94919968
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172",-86.77030181884766,34.86479949951172
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,"-91.254898, 35.6087",-91.254898,35.6087
5,00AS,small_airport,Fulton Airport,1100,NA,US,US-OK,Alex,00AS,None,00AS,"-97.8180194, 34.9428028",-97.8180194,34.9428028
6,00AZ,small_airport,Cordes Airport,3810,NA,US,US-AZ,Cordes,00AZ,None,00AZ,"-112.16500091552734, 34.305599212646484",-112.16500091552734,34.305599212646484
7,00CA,small_airport,Goldstone /Gts/ Airport,3038,NA,US,US-CA,Barstow,00CA,None,00CA,"-116.888000488, 35.350498199499995",-116.888000488,35.350498199499995
8,00CL,small_airport,Williams Ag Airport,87,NA,US,US-CA,Biggs,00CL,None,00CL,"-121.763427, 39.427188",-121.763427,39.427188
9,00CN,heliport,Kitchen Creek Helibase Heliport,3350,NA,US,US-CA,Pine Valley,00CN,None,00CN,"-116.4597417, 32.7273736",-116.4597417,32.7273736


In [49]:
df3.where(df3.longitude.isNull()).limit(20).toPandas().head(10)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,longitude,latitude


In [50]:
df3.where(df3.ident == '00XS').limit(20).toPandas().head(10)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,longitude,latitude
0,00XS,small_airport,L P Askew Farms Airport,3110,NA,US,US-TX,O'Donnell,00XS,None,00XS,"-101.93399810791016, 33.03340148925781",-101.93399810791016,33.03340148925781


In [51]:
df3.where(df3.iata_code == 'PHL').limit(20).toPandas().head(10)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,longitude,latitude
0,KPHL,large_airport,Philadelphia International Airport,36,NA,US,US-PA,Philadelphia,KPHL,PHL,PHL,"-75.24109649658203, 39.87189865112305",-75.24109649658203,39.87189865112305
1,PHL,closed,Erase Me 19,None,NA,US,US-PA,None,None,PHL,None,"-0.4, 0",-0.4,0


In [ ]:
df3.createOrReplaceTempView("airport_table")
airport_table = spark.sql(
    """
    select 
    iata_code,
    count(iata_code) as count
    from airport_table
    where type != 'closed' 
    and iso_country = 'US'
    group by iata_code
    having count(iata_code) > 1
    
    """
)

In [52]:
df3.createOrReplaceTempView("airport_table")
airport_table = spark.sql(
    """
    select 
        ident
        ,type
        ,name
        ,elevation_ft
        ,continent
        ,iso_country
        ,iso_region
        ,municipality
        ,gps_code
        ,iata_code
        ,local_code
        ,coordinates
        ,cast(longitude as decimal(8,2)) as longitude
        ,latitude as latitude2
        ,cast(RTRIM(LTRIM(latitude)) as decimal(8,2)) as latitude
        --,state_code
    from airport_table
    where type != 'closed' 
    and iso_country = 'US'
    
    """
    
    )

In [53]:
airport_table.limit(20).toPandas().head(5)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,longitude,latitude2,latitude
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125",-74.93,40.07080078125,40.07
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022",-101.47,38.704022,38.70
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968",-151.70,59.94919968,59.95
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172",-86.77,34.86479949951172,34.86
4,00AS,small_airport,Fulton Airport,1100,NA,US,US-OK,Alex,00AS,None,00AS,"-97.8180194, 34.9428028",-97.82,34.9428028,34.94


In [54]:
airport_table.show(5)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+---------+------------------+--------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|longitude|         latitude2|latitude|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+---------+------------------+--------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|   -74.93|    40.07080078125|   40.07|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|  -101.47|         38.704022|   38.70|
| 00AK|small_airport|        Lowell Field|   

In [55]:
df3 = df3.withColumn("state_code", F.split(df3.iso_region, "-").getItem(1))

In [56]:
df3.show(5)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+------------------+------------------+----------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|         longitude|          latitude|state_code|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+------------------+------------------+----------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|-74.93360137939453|    40.07080078125|        PA|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|       -101.473911|         38.704022|  

In [57]:
df_globe_temp.where((df_globe_temp.Country == "United States") & (df_globe_temp.dt >= '2013-01-01') & (df_globe_temp.City == 'Philadelphia')).limit(10).toPandas().head(10)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,2013-01-01,2.252,0.196,Philadelphia,United States,39.38N,74.91W
1,2013-02-01,1.662,0.25,Philadelphia,United States,39.38N,74.91W
2,2013-03-01,4.295,0.19899999999999998,Philadelphia,United States,39.38N,74.91W
3,2013-04-01,11.459000000000001,0.319,Philadelphia,United States,39.38N,74.91W
4,2013-05-01,16.637999999999998,0.184,Philadelphia,United States,39.38N,74.91W
5,2013-06-01,22.213,0.23600000000000002,Philadelphia,United States,39.38N,74.91W
6,2013-07-01,25.683000000000003,0.233,Philadelphia,United States,39.38N,74.91W
7,2013-08-01,22.476999999999997,0.381,Philadelphia,United States,39.38N,74.91W
8,2013-09-01,19.444000000000006,1.148,Philadelphia,United States,39.38N,74.91W


In [57]:
df_globe_temp.where(F.dayofmonth(df_globe_temp.dt) != '01').show(5)

+---+------------------+-----------------------------+----+-------+--------+---------+
| dt|AverageTemperature|AverageTemperatureUncertainty|City|Country|Latitude|Longitude|
+---+------------------+-----------------------------+----+-------+--------+---------+
+---+------------------+-----------------------------+----+-------+--------+---------+



In [58]:
df_globe_temp.createOrReplaceTempView('temp_table')

In [59]:
temp_table = spark.sql(
    """
    select  month(dt) as month 
                ,cast(avg(AverageTemperature) as decimal(16,4)) as AverageTemperatureOnMonth
                ,cast(avg(AverageTemperatureUncertainty) as decimal(16,4)) as AverageTemperatureUncertaintyOnMonth
                ,City
                ,Country
                
        From temp_table
        where dt >= '2010-01-01' 
        and City = 'Philadelphia'
        group by month(dt)
                ,City
                ,Country
    """

)

In [60]:
temp_table.show(5)

+-----+-------------------------+------------------------------------+------------+-------------+
|month|AverageTemperatureOnMonth|AverageTemperatureUncertaintyOnMonth|        City|      Country|
+-----+-------------------------+------------------------------------+------------+-------------+
|    7|                  26.0130|                              0.2825|Philadelphia|United States|
|    6|                  22.7633|                              0.3540|Philadelphia|United States|
|    1|                   1.2320|                              0.3000|Philadelphia|United States|
|    2|                   2.3510|                              0.3295|Philadelphia|United States|
|    3|                   7.3053|                              0.1865|Philadelphia|United States|
+-----+-------------------------+------------------------------------+------------+-------------+
only showing top 5 rows



In [61]:
#temperature was average per month of the year. i.e what is the average temp in jan, whats it in feb etc.
def process_avg_temp_data(spark, input_data, output_data):
    df_globe_temp = spark.read.csv(input_data, header = True)
    df_globe_temp.createOrReplaceTempView('temp_table')
    avg_temp_table = spark.sql(
        """
        select  month(dt) as month 
                ,cast(avg(AverageTemperature) as decimal(16,4)) as AverageTemperatureOnMonth
                ,cast(avg(AverageTemperatureUncertainty) as decimal(16,4)) as AverageTemperatureUncertaintyOnMonth
                ,City
                ,Country
                
        From temp_table
        where dt >= '2010-01-01' 
        group by month(dt)
                ,City
                ,Country
        
        """
    )
    
    return avg_temp_table
    

In [62]:
df_globe_temp.where((df_globe_temp.Country == "United States") & (df_globe_temp.dt >= '2013-01-01')).limit(10).toPandas().head(10)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,2013-01-01,6.32,0.267,Abilene,United States,32.95N,100.53W
1,2013-02-01,8.116,0.222,Abilene,United States,32.95N,100.53W
2,2013-03-01,12.503,0.273,Abilene,United States,32.95N,100.53W
3,2013-04-01,15.752999999999998,0.342,Abilene,United States,32.95N,100.53W
4,2013-05-01,22.545,0.125,Abilene,United States,32.95N,100.53W
5,2013-06-01,27.831,0.24600000000000002,Abilene,United States,32.95N,100.53W
6,2013-07-01,27.421,0.18100000000000002,Abilene,United States,32.95N,100.53W
7,2013-08-01,28.42,0.36200000000000004,Abilene,United States,32.95N,100.53W
8,2013-09-01,25.791,1.18,Abilene,United States,32.95N,100.53W
9,2013-01-01,-1.086,0.22,Akron,United States,40.99N,80.95W


In [63]:
#df_immigration.arrdate conversion to datetime

def convert_datetime(x):
    try:
        start = datetime(1960, 1, 1)
        return start + timedelta(days=int(x))
    except:
        return None
    
#udf_datetime_conversion = udf(lambda x: convert_datetime(x), T.DateType())
#...In your Spark code
#spark...load the data etc.... \
#   .withColumn("arrival_date", udf_datetime_from_sas("arrdate")) \
#  .withColumn("departure_date", udf_datetime_from_sas("depdate"))

In [ ]:
###create airport table
#check for duplicate ident (drop_duplicates)
#extract state code from iso_region
#extract laitude and logitude from co-ordinates table
#write updated aiport data to output path/table

In [ ]:
###create global temp table
#remove all rows with none temp
#drop duplicates (select only temperatur of the most recent date for each city)
#write global temp to output path/table

In [ ]:
###create city table
#select into a dataframe all columns except race related columns (race, race count). then remove duplicates
#pivot race column with count as value for each race column
#join both data frames above on city and state
#write city dataframe to output path/table


In [64]:
def process_airport_data(spark, input_data, output_path):
    df_airport = spark.read.csv(input_data, header = True)
    df_airport = df_airport.withColumn("longitude", F.split(df_airport.coordinates, ",").getItem(0))
    df_airport = df_airport.withColumn("latitude", F.split(df_airport.coordinates, ",").getItem(1))
    df_airport = df_airport.withColumn("state_code", F.split(df_airport.iso_region, "-").getItem(1))
    
    df_airport.createOrReplaceTempView("airport_table")
    airport_table = spark.sql(
    """
    select 
        ident
        ,type
        ,name
        ,elevation_ft
        ,continent
        ,iso_country
        ,iso_region
        ,municipality
        ,gps_code
        ,iata_code
        ,local_code
        ,cast(longitude as decimal(4,2)) as longitude
        ,cast(latitude as decimal(4,2)) as latitude
        ,state_code
    from airport_table
    where type != 'closed' 
    and iso_country = 'US'
    order by state_code
    
    """
    )
    
    airport_table.write.mode("overwrite").partitionBy("i94yr", "i94mon").parquet(output_path + "airport_dimension_data")
    #return airport_table
    
    
    
    
    

In [65]:
def process_city_data(spark, input_data_city, input_data_city_state_code, output_path):
    
    #read in data files
    df_cities = spark.read.option("delimiter", ";").csv(input_data_city, header = True)
    df_city_state_code = spark.read.csv(input_data_city_state_code, header = True)
    
    #remove space and '-' from column names
    df_cities2 = df_cities.select([F.col(col).alias(col.replace(' ', '_')) for col in df_cities.columns])
    df_cities2 = df_cities2.select([F.col(col).alias(col.replace('-', '_')) for col in df_cities2.columns])
    
    #select and pivot race column from city data frame to get a single race row per city
    df_race = df_cities2.select(df_cities2.city_state_code, df_cities2.Race, df_cities2.Count.cast('int'))
    df_race = df_race.groupBy("city_state_code").pivot("Race").sum("Count")
    
    #concat city name and state code
    df_cities2 = df_cities2.withColumn("city_state_code", F.concat_ws(('_'), F.col(("city")), F.col(("State_Code"))))
    df_city_state_code = df_city_state_code.withColumn("city_state_code", F.concat_ws(('_'), F.col(("city_name")), F.col(("state_code"))))
    
    #create temp table for all 3 data frames
    df_cities2.createOrReplaceTempView("cities")
    df_race.createOrReplaceTempView("race_table")
    df_city_state_code.createOrReplaceTempView("city_code")
    
    #join all 3 tables to get single row per city with complete city code
    city_state_code = spark.sql(
    """
    select e.City
            ,e.State
            ,cc.city_code
            ,e.Median_Age
            ,cast(e.Male_Population as int) as Male_Population
            ,cast(e.Female_Population as int) as Female_Population
            ,cast(e.Total_Population as int) as Total_Population
            ,cast(e.Number_of_Veterans as int) as Number_of_Veterans
            ,cast(e.Foreign_born as int) as Foreign_born
            ,e.Average_Household_Size
            ,e.State_Code
            ,r.*
            
    from (select 
            City
            ,State
            ,city_state_code
            ,avg(Median_Age) as Median_Age
            ,sum(Male_Population) as Male_Population
            ,sum(Female_Population) as Female_Population
            ,sum(Total_Population) as Total_Population
            ,sum(Number_of_Veterans) as Number_of_Veterans
            ,sum(Foreign_born) as Foreign_born
            ,avg(Average_Household_Size) as Average_Household_Size
            ,State_Code
            --,r.*
            --,Race
            --,Count
            from cities
    group by city_state_code, City, State, State_Code ) as e
    inner join race_table r on e.city_state_code = r.city_state_code
    inner join city_code cc on lower(e.city_state_code) = lower(cc.city_state_code)
    
    """ 
    )
    
    city_state_code.write.mode("overwrite").parquet(output_path + "city_dimension_data")
    
    #return city_state_code
    
    
    #input_data_city = "us-cities-demographics.csv"
    #input_data_city_state_code = "us_cities_processed.csv"
    
    
    
    
    
    
    
    

In [66]:
def fn_format_datetime(x):
    try:
        start = datetime(1960, 1, 1)
        return start + timedelta(days=int(x))
    except:
        return None
    
spark.udf.register("format_datetime", fn_format_datetime, T.DateType())

<function __main__.fn_format_datetime(x)>

In [67]:
def processed_immigration_data(spark, input_data, output_path):
    df = spark.read.parquet(input_data)
    df.createOrReplaceTempView("immigration_data")
    
    immigration_facts_table = spark.sql(
    """
    select  cast(cicid as int) as cicid
            ,cast(i94yr as int) as i94yr
            ,cast(i94mon as int) as i94mon
            ,cast(i94cit as int) as i94cit
            ,cast(i94res as int) as i94res
            ,i94port
            ,format_datetime(cast(arrdate as int)) as arrival_date
            ,cast(i94mode as int) as i94mode
            ,i94addr
            ,format_datetime(cast(depdate as int)) as depature_date
            ,cast(i94bir as int) as i94bir
            ,cast(i94visa as int) as i94visa
            ,cast(count as int) as count
            ,to_date(cast(UNIX_TIMESTAMP(dtadfile, 'MMddyyyy') AS TIMESTAMP)) AS dtadfile
            ,visapost
            ,occup
            ,entdepa
            ,entdepd
            ,entdepu
            ,matflag
            ,cast(biryear as int) as biryear
            ,to_date(cast(UNIX_TIMESTAMP(dtaddto, 'MMddyyyy') AS TIMESTAMP)) AS dtaddto
            ,gender
            ,insnum
            ,airline
            ,cast(admnum as decimal(16,8)) as admnum
            ,fltno
            ,visatype
            
    From immigration_data
    
    """ )
    
    immigration_table.write.mode("overwrite").partitionBy("i94yr", "i94mon").parquet(output_path + "immigration_facts_data")
    
    #return immigration_table
    
    
    
    

In [68]:
def processed_visa_type(spark, input_data, output_data):
    df_visa = spark.read.csv(input_data, header = True)
    
    df_visa.write.mode("overwrite").parquet(output_path + "visa_type_dimension_data")
    
    #return df_visa

In [69]:
def processed_transport_mode(spark, input_data, output_path):
    df_transport_mode = spark.read.csv(input_data, header = True)
    
    df_transport_mode.write.mode("overwrite").parquet(output_path + "transport_mode_dimension_data")
    
    #return df_transport_mode

In [70]:
#temperature was average per month of the year. i.e what is the average temp in jan, whats it in feb etc.
def process_avg_temp_data(spark, input_path, input_data_city_state_code, output_path):
    
    df_globe_temp = spark.read.csv(input_path, header = True)
    df_city_state_code = spark.read.csv(input_data_city_state_code, header = True)
    
    df_globe_temp.createOrReplaceTempView('temp_table')
    df_city_state_code.createOrReplaceTempView("city_code")
    avg_temp_table = spark.sql(
        """
        select  month(dt) as month 
                ,cast(avg(AverageTemperature) as decimal(16,4)) as AverageTemperatureOnMonth
                ,cast(avg(AverageTemperatureUncertainty) as decimal(16,4)) as AverageTemperatureUncertaintyOnMonth
                --,City
                ,c.city_code
                --,Country
                
        From temp_table t
        left join city_code c on lower(t.City) = lower(c.city_name)
        where dt >= '2010-01-01' 
        group by month(dt)
                ,City
                ,Country
        
        """
    )
    
    avg_temp_table.write.mode("overwrite").partitionBy("month").parquet(output_path + "average_temp_dimension_data")
    
    #return avg_temp_table
    
    

In [ ]:
def data_copy_quality_check(spark, input_source, dest_path):
    
    #read source tables
    df_cities_source = spark.read.option("delimiter", ";").csv(input_source + "us-cities-demographics.csv", header = True)
    df_cities_source = df_cities_source.select([F.col(col).alias(col.replace(' ', '_')) for col in df_cities_source.columns])
    df_visa_source = spark.read.csv(input_source + "visa_type.csv", header = True)
    df_airport_source = spark.read.csv(input_source + "/airport-codes_csv.csv", header = True)
    df_globe_temp_source = spark.read.csv(global_temp_path, header = True)
    df_country_code_source = spark.read.csv(input_source + "/country_code_processed.csv", header = True)
    df_transport_mode_source = spark.read.csv(input_source + "/transport_mode.csv", header = True)
    df_immigration_source = spark.read.parquet(input_source + "/sas_data")
    
    #create source temp tables
    df_cities_source = df_cities_source.createOrReplaceTempView('city_source_table')
    df_visa_source = df_visa_source.createOrReplaceTempView('visa_source_table')
    df_airport_source = df_airport_source.createOrReplaceTempView('airport_source_table')
    df_globe_temp_source = df_globe_temp_source.createOrReplaceTempView('temp_source_table')
    df_country_code_source = df_country_code_source.createOrReplaceTempView('country_source_table')
    df_transport_mode_source = df_transport_mode_source.createOrReplaceTempView('transport_source_table')
    df_immigration_source = df_immigration_source.createOrReplaceTempView('immigration_source_table')
    
    
    #read destination tables
    df_cities_dest = spark.read.parquet(dest_path + "/city_dimension_data")
    df_visa_dest = spark.read.parquet(dest_path + "/visa_dimension_data")
    df_airport_dest = spark.read.parquet(dest_path + "/airport_dimension_data")
    df_globe_temp_dest = spark.read.parquet(dest_path + "/us_temperature_dimension_data")
    df_country_code_dest = spark.read.parquet(dest_path + "/country_dimension_data")
    df_transport_mode_dest = spark.read.parquet(dest_path + "/transport_mode_dimension_data")
    df_immigration_dest = spark.read.parquet(dest_path + "/immigration_facts_data")
    
    
    #create destination temp tables
    df_cities_dest = df_cities_dest.createOrReplaceTempView('city_dest_table')
    df_visa_dest = df_visa_dest.createOrReplaceTempView('visa_dest_table')
    df_airport_dest = df_airport_dest.createOrReplaceTempView('airport_dest_table')
    df_globe_temp_dest = df_globe_temp_dest.createOrReplaceTempView('temp_dest_table')
    df_country_code_dest = df_country_code_dest.createOrReplaceTempView('country_dest_table')
    df_transport_mode_dest = df_transport_mode_dest.createOrReplaceTempView('transport_dest_table')
    df_immigration_dest = df_immigration_dest.createOrReplaceTempView('immigration_dest_table')
    
    row_count_check = spark.sql(
        """
        select table_name,
                sum(source_count) as source_count,
                sum(destination_count) as destination_count
        from (
        select 'city_table' as table_name,
                count(distinct (City + '_'+State_Code)) as source_count,
                0 as destination_count
        from city_source_table
        
        union
        
        select 'city_table' as table_name,
                0 as source_count,
                count(city_state_code) as destination_count
        from city_dest_table
        
        
        
        ----
        union
        
        select 'visa_table' as table_name,
                count(visa_type_id) as source_count,
                0 as destination_count
        from visa_source_table
        
        union
        
        select 'visa_table' as table_name,
                0 as source_count,
                count(visa_type_id) as destination_count
        from visa_dest_table
        
        
        
        
        
        ----
        
        union
        
        select 'airport_table' as table_name,
                count(iata_code) as source_count,
                0 as destination_count
        from airport_source_table
        where type != 'closed' 
              and iso_country = 'US'
        
        union
        
        select 'airport_table' as table_name,
                0 as source_count,
                count(iata_code) as destination_count
        from airport_dest_table
        
        
        
        ----
        union
        
        select 'country_table' as table_name,
                count(country_code) as source_count,
                0 as destination_count
        from country_dest_table
        
        union
        
        select 'country_table' as table_name,
                0 as source_count,
                count(country_code) as destination_count
        from country_dest_table
        
        
        
        
        ----
        union
        
        select 'transport_table' as table_name,
                count(trans_mode_id) as source_count,
                0 as destination_count
        from transport_source_table
        
        union
        
        select 'transport_table' as table_name,
                0 as source_count,
                count(trans_mode_id) as destination_count
        from transport_dest_table
        
        
        ----
         union
        
        select 'immigration_table' as table_name,
                count(cicid) as source_count,
                0 as destination_count
        from immigration_source_table
        
        union
        
        select 'immigration_table' as table_name,
                0 as source_count,
                count(cicid) as destination_count
        from immigration_dest_table
        )
        group by table_name
        
        """    
    )
    row_count_check.write.mode("overwrite").parquet(dest_path + "row_count_check")

    
    
    

In [3]:
data_quality_check = spark.read.parquet("spark-warehouse/row_count_check")

In [4]:
data_quality_check.show(10)

+-----------------+------------+-----------------+
|       table_name|source_count|destination_count|
+-----------------+------------+-----------------+
|immigration_table|     3096313|          3096313|
|  transport_table|           4|                4|
|    country_table|         289|              289|
|    airport_table|        1956|             1956|
|       visa_table|           3|                3|
|       city_table|         567|              567|
+-----------------+------------+-----------------+



In [3]:
data_quality_check2 = spark.read.parquet("spark-warehouse/duplicate_row_check")

In [4]:
data_quality_check2.show()

+-------------+--------------------+
|   table_name|number_of_duplicates|
+-------------+--------------------+
|airport_table|                   0|
|   city_table|                  28|
+-------------+--------------------+



In [32]:
source_city = spark.read.parquet("spark-warehouse/source_city")

In [33]:
dest_city = spark.read.parquet("spark-warehouse/dest_city")

In [34]:
source_city.repartition(1).write.mode("overwrite").csv("spark-warehouse/source_city2")

In [35]:
dest_city.repartition(1).write.mode("overwrite").csv("spark-warehouse/dest_city2")

In [91]:
airport_dimension_data = spark.read.parquet("spark-warehouse/airport_dimension_data")

In [92]:
airport_dimension_data2   = airport_dimension_data.toPandas()

In [80]:
airport_dimension_data.describe()

DataFrame[summary: string, ident: string, type: string, name: string, elevation_ft: string, continent: string, iso_country: string, iso_region: string, municipality: string, gps_code: string, iata_code: string, local_code: string, longitude: string, latitude: string, state_code: string]

In [93]:
airport_dimension_data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21431 entries, 0 to 21430
Data columns (total 14 columns):
ident           21431 non-null object
type            21431 non-null object
name            21431 non-null object
elevation_ft    21231 non-null object
continent       21431 non-null object
iso_country     21431 non-null object
iso_region      21431 non-null object
municipality    21364 non-null object
gps_code        20881 non-null object
iata_code       1956 non-null object
local_code      21125 non-null object
longitude       21431 non-null object
latitude        21431 non-null object
state_code      21431 non-null object
dtypes: object(14)
memory usage: 2.3+ MB


In [95]:
city_dimension_data = spark.read.parquet("spark-warehouse/city_dimension_data")

In [96]:
city_dimension_data = city_dimension_data.toPandas()

In [98]:
city_dimension_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 17 columns):
City                                 115 non-null object
State                                115 non-null object
city_code                            115 non-null object
Median_Age                           115 non-null float64
Male_Population                      115 non-null int32
Female_Population                    115 non-null int32
Total_Population                     115 non-null int32
Number_of_Veterans                   113 non-null float64
Foreign_born                         113 non-null float64
Average_Household_Size               113 non-null float64
State_Code                           115 non-null object
city_state_code                      115 non-null object
American_Indian_and_Alaska_Native    108 non-null float64
Asian                                113 non-null float64
Black_or_African_American            112 non-null float64
Hispanic_or_Latino                  

In [99]:
country_dimension_data = spark.read.parquet("spark-warehouse/country_dimension_data")

In [100]:
country_dimension_data = country_dimension_data.toPandas()

In [102]:
country_dimension_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289 entries, 0 to 288
Data columns (total 2 columns):
country_code    289 non-null object
country_name    289 non-null object
dtypes: object(2)
memory usage: 4.6+ KB


In [103]:
immigration_facts_data = spark.read.parquet("spark-warehouse/immigration_facts_data")

In [105]:
immigration_facts_data = immigration_facts_data.limit(100000).toPandas()

In [106]:
immigration_facts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
cicid            100000 non-null int32
i94cit           100000 non-null int32
i94res           100000 non-null int32
i94port          100000 non-null object
arrival_date     100000 non-null object
i94mode          99992 non-null float64
i94addr          93715 non-null object
depature_date    94005 non-null object
i94bir           99964 non-null float64
i94visa          100000 non-null int32
count            100000 non-null int32
dtadfile         0 non-null object
visapost         43433 non-null object
occup            220 non-null object
entdepa          99992 non-null object
entdepd          94210 non-null object
entdepu          9 non-null object
matflag          94210 non-null object
biryear          99964 non-null float64
dtaddto          99185 non-null object
gender           87934 non-null object
insnum           2076 non-null object
airline          98045 non-null object

In [107]:
transport_mode_dimension_data = spark.read.parquet("spark-warehouse/transport_mode_dimension_data")

In [108]:
transport_mode_dimension_data = transport_mode_dimension_data.toPandas()

In [116]:
transport_mode_dimension_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
trans_mode_id      4 non-null object
trans_mode_name    4 non-null object
dtypes: object(2)
memory usage: 144.0+ bytes


In [110]:
us_temperature_dimension_data = spark.read.parquet("spark-warehouse/us_temperature_dimension_data")

In [111]:
us_temperature_dimension_data = us_temperature_dimension_data.toPandas()

In [112]:
us_temperature_dimension_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1164 entries, 0 to 1163
Data columns (total 4 columns):
AverageTemperatureOnMonth               1164 non-null object
AverageTemperatureUncertaintyOnMonth    1164 non-null object
city_code                               1152 non-null object
month                                   1164 non-null int32
dtypes: int32(1), object(3)
memory usage: 31.9+ KB


In [113]:
visa_dimension_data = spark.read.parquet("spark-warehouse/visa_dimension_data")

In [114]:
visa_dimension_data = visa_dimension_data.toPandas()

In [115]:
visa_dimension_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
visa_type_id      3 non-null object
visa_type_name    3 non-null object
dtypes: object(2)
memory usage: 128.0+ bytes
